<a href="https://colab.research.google.com/github/JoseJ09/Feature-Engineeering-Chicago-Inspections/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalar libreria
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.2 MB/s eta 0:00:00


## Feature Engineeering Chicago Inspections

In [ ]:
#Importar librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import boto3
import yaml

from datetime import date
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

### 1. Cargar los datos ingestados del bucket de S3

In [ ]:
#Obtener las credenciales del archivo yaml
with open("credentials.yaml", "r") as f:
    config = yaml.safe_load(f)

In [ ]:
#Funcion para cargar datos del bucket
def cargar_datos_s3(bucket, bucket_path):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key'],
        aws_session_token = config['s3']['aws_session_token']
    )

    s3 = session.resource('s3')

    obj = s3.Object(bucket, bucket_path).get()['Body'].read()
    dataset = pickle.loads(obj)

    return dataset

In [ ]:
#Iniciamos sesión en AWS
session = boto3.Session(
    aws_access_key_id = config['s3']['aws_access_key_id'],
    aws_secret_access_key = config['s3']['aws_secret_access_key'],
    aws_session_token = config['s3']['aws_session_token']
)

s3 = session.client('s3')

In [ ]:
#Seleccionar bucket y ruta
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "limpieza/"

In [ ]:
#Obtener path del bucket
bucket_path = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents'][-1]['Key']

In [ ]:
#Mostrar path
bucket_path

'limpieza/datos-limpios-2023-04-21.pkl'

In [ ]:
#Se cargan los datos de la limpieza de datos
dataset = cargar_datos_s3(bucket, bucket_path)

### 2. Eliminar observaciones con resultados inválidos

In [ ]:
#Como se ven los primeros registros
dataset.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"{'latitude': '41.89843137207629', 'longitude':..."
1,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
2,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
3,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,License Re-Inspection,Pass,NaN,41.938443,-87.768318,"{'latitude': '41.93844282365204', 'longitude':..."
4,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"{'latitude': '41.938006880423615', 'longitude'..."


In [ ]:
#Comprobamos los tipos de datos
dataset.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
violations                 object
latitude                  float64
longitude                 float64
location                   object
dtype: object

In [ ]:
#Agrupamos por tipo de resultados y contamos la cantidad de id
dataset.groupby(['results'], as_index=False)['inspection_id'].count()

,results,inspection_id
0,Business Not Located,78
1,Fail,48769
2,No Entry,9742
3,Not Ready,3058
4,Out of Business,21482
5,Pass,129368
6,Pass w/ Conditions,38677


In [ ]:
#Mostramos el dataset
dataset

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"{'latitude': '41.89843137207629', 'longitude':..."
1,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
2,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
3,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,License Re-Inspection,Pass,NaN,41.938443,-87.768318,"{'latitude': '41.93844282365204', 'longitude':..."
4,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"{'latitude': '41.938006880423615', 'longitude'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252038,2573792,TACOS BUSCADOS,TACOS BUSCADOS,2895701,Restaurant,Risk 1 (High),7955 W ADDISON ST,CHICAGO,IL,60634,2023-04-06,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.944923,-87.826448,"{'latitude': '41.94492330711078', 'longitude':..."
252039,2573813,IVY GARDEN COMMUNITY CENTER,IVY GARDEN LEARNING CENTER,2835986,ADULT DAYCARE,Risk 1 (High),2929 S HALSTED ST,CHICAGO,IL,60608,2023-04-06,Canvass,Pass,NaN,41.840791,-87.646198,"{'latitude': '41.84079081861273', 'longitude':..."
252040,2573815,ORIOLE,ORIOLE,2781335,Restaurant,Risk 1 (High),661 W WALNUT ST,CHICAGO,IL,60661,2023-04-06,Non-Inspection,No Entry,NaN,41.886135,-87.645175,"{'latitude': '41.886135368620835', 'longitude'..."
252041,2573794,SEOUL TACO,SEOUL TACO,2862550,Restaurant,Risk 1 (High),1422 N MILWAUKEE AVE,CHICAGO,IL,60622,2023-04-06,Canvass,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.907715,-87.673529,"{'latitude': '41.90771541543215', 'longitude':..."


In [ ]:
#Funcion para filtrar resultados
def delete_inspections(df):
    return df[df.results.isin(['Fail', 'Not Ready', 'Pass', 'Pass w/ Conditions'])]

In [ ]:
#Creamos el dataset con resultados filtrados
inspections_wo_not_valid_results = delete_inspections(dataset)

In [ ]:
#Revisamos columnas y registros
inspections_wo_not_valid_results.shape

(219872, 17)

In [ ]:
#Contamos registros unicos
inspections_wo_not_valid_results.results.value_counts()

Pass                  129368
Fail                   48769
Pass w/ Conditions     38677
Not Ready               3058
Name: results, dtype: int64

### 3. Modificar etiquetas de resultados

In [ ]:
#Función para modificar etiquetas , dejamos solo como pass y en fail
def transform_result(df):
    df.results.mask(df.results.isin(['Pass w/ Conditions', 'Pass']), 'pass', inplace=True)
    df.results.mask(df.results.isin(['Fail', 'Not Ready']), 'fail', inplace=True)

    return df

In [ ]:
#Guardamos el dataset transformado
inspections = transform_result(inspections_wo_not_valid_results)

In [ ]:
inspections

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04,Canvass,fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"{'latitude': '41.89843137207629', 'longitude':..."
1,67732,WOLCOTT'S,TROQUET,1992039,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
2,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,pass,NaN,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'..."
3,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,License Re-Inspection,pass,NaN,41.938443,-87.768318,"{'latitude': '41.93844282365204', 'longitude':..."
4,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,License Re-Inspection,pass,NaN,41.938007,-87.644755,"{'latitude': '41.938006880423615', 'longitude'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252036,2573799,FLOWERS NUTRITION LLC,FLOWERS NUTRITION LLC,2873170,Restaurant,Risk 2 (Medium),3332 W MONTROSE AVE,CHICAGO,IL,60618,2023-04-06,License,fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.961251,-87.712052,"{'latitude': '41.96125126209583', 'longitude':..."
252037,2573797,SEOUL TACO,SEOUL TACO,2891250,Restaurant,Risk 3 (Low),1422 N MILWAUKEE AVE,CHICAGO,IL,60622,2023-04-06,License,pass,NaN,41.907715,-87.673529,"{'latitude': '41.90771541543215', 'longitude':..."
252038,2573792,TACOS BUSCADOS,TACOS BUSCADOS,2895701,Restaurant,Risk 1 (High),7955 W ADDISON ST,CHICAGO,IL,60634,2023-04-06,License,fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.944923,-87.826448,"{'latitude': '41.94492330711078', 'longitude':..."
252039,2573813,IVY GARDEN COMMUNITY CENTER,IVY GARDEN LEARNING CENTER,2835986,ADULT DAYCARE,Risk 1 (High),2929 S HALSTED ST,CHICAGO,IL,60608,2023-04-06,Canvass,pass,NaN,41.840791,-87.646198,"{'latitude': '41.84079081861273', 'longitude':..."


In [ ]:
#Contamos valores unicos por resultado
inspections.results.value_counts()

pass    168045
fail     51827
Name: results, dtype: int64

### 4. Modificar etiquetas de riesgo

In [ ]:
#Modificamos etiquetas de riesgo, las convertimos a minusculas
def transform_risk(df):
    df.risk.mask(df.risk.str.contains('Low'), 'low', inplace=True)
    df.risk.mask(df.risk.str.contains('Medium'), 'medium', inplace=True)
    df.risk.mask(df.risk.str.contains('High'), 'high', inplace=True)
    df.risk.mask(df.risk.str.contains('All'), 'all', inplace=True)

    return df

In [ ]:
#Guardamos el dataset transformado
inspections = transform_risk(inspections)

<ipython-input-22-68a7e53f1b5b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.risk.mask(df.risk.str.contains('Low'), 'low', inplace=True)
<ipython-input-22-68a7e53f1b5b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.risk.mask(df.risk.str.contains('Medium'), 'medium', inplace=True)
<ipython-input-22-68a7e53f1b5b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.risk.mask(df.risk.str.contains('High'), 'high', inplace=True)

### 5. Modificar las etiquetas de facility_type

In [ ]:
#Modificamos etiquetas de facility type, las convertimos a minusculas
def transform_facility(df):
    df.facility_type.mask(df.facility_type.str.contains('Restaurant'), 'restaurant', inplace=True)
    df.facility_type.mask(df.facility_type.str.contains('Daycare'), 'daycare', inplace=True)
    df.facility_type.mask(df.facility_type.str.contains('Mobile Food'), 'mobile food', inplace=True)

    return df

In [ ]:
#Guardamos el dataset transformado
inspections = transform_facility(inspections)

<ipython-input-24-8cba8d7a2118>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.facility_type.mask(df.facility_type.str.contains('Restaurant'), 'restaurant', inplace=True)
<ipython-input-24-8cba8d7a2118>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.facility_type.mask(df.facility_type.str.contains('Daycare'), 'daycare', inplace=True)
<ipython-input-24-8cba8d7a2118>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.facility_

In [ ]:
#Creamos auxiliar agrupado para filtrar posteriormente, contamos registros, renombramos columnas y mostramos primeros 20
aux = inspections.groupby(['facility_type'], as_index=False)['inspection_id']\
.count()\
.rename(columns={'inspection_id': 'count'})\
.sort_values(by="count", ascending=False).head(20)

In [ ]:
#Función para modificar etiquetas que no estan en nuestro aux como 'other', las convertimos a minusculas
def transform_facility_other(df):
    df.facility_type.mask(~(df.facility_type.isin(aux.facility_type.values)), 'other', inplace=True)

    return df

In [ ]:
#Guardamos el dataset transformado
inspections = transform_facility_other(inspections)

<ipython-input-27-506b2e01a156>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.facility_type.mask(~(df.facility_type.isin(aux.facility_type.values)), 'other', inplace=True)


In [ ]:
#Mostramos los primeros 30 regsitros de nuestro dataset con los cambios realizados
inspections.groupby(['facility_type'], as_index=False)['inspection_id'].count()\
.sort_values(by="inspection_id", ascending=False).head(30)

,facility_type,inspection_id
20,restaurant,149502
6,Grocery Store,28108
10,School,15047
17,daycare,6686
3,Children's Services Facility,4754
19,other,3815
1,Bakery,3195
9,Long Term Care,1708
18,mobile food,1550
2,Catering,1277


### 6. Creación de features

In [ ]:
#Función para generar features por tiempo: Mes, año, dia del mes, semana del año , dia de la semana , dia entre semana y fin de semana
def generate_features(df):
    df['month'] = df.inspection_date.dt.month
    df['year'] = df.inspection_date.dt.year
    df['day_of_month'] = df.inspection_date.dt.day
    df['week_of_year'] = df.inspection_date.dt.week
    df['day_of_week'] = df.inspection_date.dt.dayofweek
    # day of week 0: lunes
    df['week_day'] = 1
    df.week_day.mask(df.week_day.isin([5,6]), 0, inplace=True)
    df['weekend'] = 0
    df.week_day.mask(df.week_day.isin([5,6]), 1, inplace=True)

    return df

In [ ]:
#Guardamos el dataset transformado con features de fechas
inspections = generate_features(inspections)

<ipython-input-30-4a311e6b9b96>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df.inspection_date.dt.month
<ipython-input-30-4a311e6b9b96>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df.inspection_date.dt.year
<ipython-input-30-4a311e6b9b96>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [ ]:
#Mostramos primeros registros
inspections.head()

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,...,latitude,longitude,location,month,year,day_of_month,week_of_year,day_of_week,week_day,weekend
0,104236,TEMPO CAFE,TEMPO CAFE,80916,restaurant,high,6 E CHESTNUT ST,CHICAGO,IL,60611,...,41.898431,-87.628009,"{'latitude': '41.89843137207629', 'longitude':...",1,2010,4,1,0,1,0
1,67732,WOLCOTT'S,TROQUET,1992039,restaurant,high,1834 W MONTROSE AVE,CHICAGO,IL,60613,...,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'...",1,2010,4,1,0,1,0
2,67733,WOLCOTT'S,TROQUET,1992040,restaurant,high,1834 W MONTROSE AVE,CHICAGO,IL,60613,...,41.961606,-87.675967,"{'latitude': '41.961605669949854', 'longitude'...",1,2010,4,1,0,1,0
3,70269,mr.daniel's,mr.daniel's,1899292,restaurant,high,5645 W BELMONT AVE,CHICAGO,IL,60634,...,41.938443,-87.768318,"{'latitude': '41.93844282365204', 'longitude':...",1,2010,4,1,0,1,0
4,52234,Cafe 608,Cafe 608,2013328,restaurant,high,608 W BARRY AVE,CHICAGO,IL,60657,...,41.938007,-87.644755,"{'latitude': '41.938006880423615', 'longitude'...",1,2010,4,1,0,1,0


### 7. Crear la matriz de diseño

In [ ]:
#Creamos la matriz de diseño con las columnas que deseamos utilizar
df = inspections[['facility_type', 'risk', 'latitude', 'longitude', 'month', 'year', 'day_of_month',
                 'week_of_year', 'day_of_week', 'week_day', 'weekend','results']]

In [ ]:
#Creamos los transformers aplicando Onehotencoder  en facility type y risk  y dejando las columnas que no estan en transformers
transformers = [('one_hot_risk', OneHotEncoder(), ['facility_type']),
               ('one_hot_facility_type', OneHotEncoder(), ['risk'])]

ct = ColumnTransformer(transformers, remainder="passthrough", n_jobs=-1)

In [ ]:
#Corremos la transformación y guardamos en fm
fm = ct.fit_transform(df)

In [ ]:
#Revisamos registros y columnas
fm.shape

(219872, 35)

In [ ]:
#Creamos arreglo con los nombres de los features presentes en el column transformer
np.array(ct.get_feature_names_out())

array(['one_hot_risk__facility_type_BANQUET HALL',
       'one_hot_risk__facility_type_Bakery',
       'one_hot_risk__facility_type_Catering',
       "one_hot_risk__facility_type_Children's Services Facility",
       'one_hot_risk__facility_type_GAS STATION',
       'one_hot_risk__facility_type_Golden Diner',
       'one_hot_risk__facility_type_Grocery Store',
       'one_hot_risk__facility_type_Hospital',
       'one_hot_risk__facility_type_Liquor',
       'one_hot_risk__facility_type_Long Term Care',
       'one_hot_risk__facility_type_School',
       'one_hot_risk__facility_type_Shared Kitchen',
       'one_hot_risk__facility_type_Shared Kitchen User (Long Term)',
       'one_hot_risk__facility_type_Shelter',
       'one_hot_risk__facility_type_Special Event',
       'one_hot_risk__facility_type_TAVERN',
       'one_hot_risk__facility_type_Wholesale',
       'one_hot_risk__facility_type_daycare',
       'one_hot_risk__facility_type_mobile food',
       'one_hot_risk__facility_type_o

In [ ]:
#Creamos nuestra feature matrix , conviertiendo a dataframe fm con las columnas de nuestro arreglo
feature_matrix = pd.DataFrame(fm, columns=np.array(ct.get_feature_names_out()))

In [ ]:
#Mostramos primeros registros de nuestra matriz
feature_matrix.head()

,one_hot_risk__facility_type_BANQUET HALL,one_hot_risk__facility_type_Bakery,one_hot_risk__facility_type_Catering,one_hot_risk__facility_type_Children's Services Facility,one_hot_risk__facility_type_GAS STATION,one_hot_risk__facility_type_Golden Diner,one_hot_risk__facility_type_Grocery Store,one_hot_risk__facility_type_Hospital,one_hot_risk__facility_type_Liquor,one_hot_risk__facility_type_Long Term Care,...,remainder__latitude,remainder__longitude,remainder__month,remainder__year,remainder__day_of_month,remainder__week_of_year,remainder__day_of_week,remainder__week_day,remainder__weekend,remainder__results
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.898431,-87.628009,1,2010,4,1,0,1,0,fail
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,1,2010,4,1,0,1,0,pass
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.961606,-87.675967,1,2010,4,1,0,1,0,pass
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.938443,-87.768318,1,2010,4,1,0,1,0,pass
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41.938007,-87.644755,1,2010,4,1,0,1,0,pass


### 8. Guardar la matriz de diseño

In [ ]:
#Función para guardar nuestra matriz en el bucket
def save_feature_matrix(bucket, bucket_path, dataset):
    session = boto3.Session(
        aws_access_key_id = config['s3']['aws_access_key_id'],
        aws_secret_access_key = config['s3']['aws_secret_access_key'],
        aws_session_token = config['s3']['aws_session_token']
    )

    s3 = session.resource('s3')

    s3.Object(bucket, bucket_path).put(Body=dataset)

In [ ]:
#Obtenemos fecha
TODAY = date.today()

In [ ]:
#Convertimos a formato pickle los datos limpios
pickle_data = pickle.dumps(feature_matrix)

In [ ]:
#Seleccionamos bucket , definimos ruta y guardamos
bucket = "aplicaciones-cd-1-" + config['iexe']['matricula']
key = "feature-matrix/feature-matrix-" + str(TODAY) + ".pkl"
save_feature_matrix(bucket, key, pickle_data)